In [193]:
# Dependencies
import requests
import json
import gmaps
import pandas as pd
import numpy as np
from config import gkey
from pprint import pprint

In [198]:
# read csv file
file="retrieved_city_data.csv"
cities_data_df=pd.read_csv(file)
city_df=(cities_data_df.drop(['Unnamed: 0','Lng','Lat'],axis=1)).dropna()

city_df.head()

,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (m/s),Wind Speed (mph)
0,thompson,ca,-20.20,75.0,5.0,0.74,1.655336
2,muisne,ec,76.48,92.0,66.0,1.97,4.406772
3,saint-philippe,re,81.93,62.0,35.0,3.36,7.516118
5,arraial do cabo,br,75.76,85.0,0.0,10.80,24.158952
6,albany,au,73.40,56.0,20.0,12.75,28.520985


In [199]:
#get lat lng for cities
city_df['Lat']=""
city_df["Lng"]=""

# use iterrows to iterate through pandas dataframe
for index, row in cities_data_df.iterrows():
    
    target_city=row["City"]
    region=row["Country"]
    
    # target url
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&region={1}&key={2}').format(target_city,region, gkey)
    geo_data = requests.get(target_url).json()
    results = geo_data['results']
    # Extract latitude and longitude
    try:
       
        city_df.loc[index, 'Lat']= results[0]["geometry"]["location"]["lat"]
        city_df.loc[index, 'Lng']= results[0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
        
        
        

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [201]:
city_df.head(200)



city_df.dtypes


City                 object
Country              object
Temperature (F)     float64
Humidity (%)        float64
Cloudiness (%)      float64
Wind Speed (m/s)    float64
Wind Speed (mph)    float64
Lat                  object
Lng                  object
dtype: object

In [202]:
gmaps.configure(api_key=gkey)


# Convert humidity to float, store and  handle NaN values

# Store 'Lat' and 'Lng' into  locations 
city_df["Lat"] = pd.to_numeric(city_df["Lat"] , errors='coerce')
city_df["Lng"] = pd.to_numeric(city_df["Lng"] , errors='coerce')
city_df = city_df.dropna()
locations=city_df[["Lat","Lng"]]
humidity = city_df["Humidity (%)"].astype(float)

In [203]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [235]:
#define a ideal location for vacation
ideal_df= city_df.loc[( city_df["Temperature (F)"] <70.0) & (city_df["Temperature (F)"]>50.0) ]

ideal_df=ideal_df.loc[(ideal_df["Wind Speed (mph)"]<10) & (ideal_df["Cloudiness (%)"]==0) ]


# Convert humidity to float and store
# handle NaN values
ideal_df = ideal_df.dropna()
# Store 'Lat' and 'Lng' into  locations 
ideal_locations = ideal_df[["Lat", "Lng"]].astype(float)
ideal_humidity = ideal_df["Humidity (%)"].astype(float)
ideal_df.head()

ideal_df['name'] = ""
ideal_df['address'] = ""
ideal_df.head(18)

,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (m/s),Wind Speed (mph),Lat,Lng,name,address
37,sabha,ly,50.74,24.0,0.0,3.18,7.113469,27.036541,14.429024,,
194,henties bay,na,65.32,83.0,0.0,2.01,4.496249,-22.113496,14.283204,,
220,laguna,br,64.42,82.0,0.0,2.26,5.055484,-28.487606,-48.839740,,
265,qom,ir,55.78,19.0,0.0,1.59,3.556735,34.641576,50.874603,,
272,adeje,es,63.21,63.0,0.0,3.36,7.516118,28.122060,-16.733821,,
312,jalu,ly,55.60,68.0,0.0,4.07,9.104346,29.040621,21.499133,,
501,hokitika,nz,63.70,71.0,0.0,3.29,7.359533,-42.717369,170.966543,,


In [236]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(ideal_locations, weights=ideal_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [206]:
# find the closest hotel of each type to coordinates


target_radius = 5000
target_type = "lodging"
# set up a parameters dictionary
params = {
    
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"



In [237]:
# use iterrows to iterate through pandas dataframe
for index, row in  ideal_df.iterrows():

    # add keyword to params dict
    params['location'] =f"{row['Lat']},{row['Lng']}"

    # assemble url and make API request
    #print(f" Retrieving lodging for Index {index} : {row['Lat']}, {row['Lng']}.")
    response = requests.get(base_url, params=params).json()
    print(json.dumps(response, indent=4, sort_keys=True))
    # extract results
    results = response['results']
    
    try:
        print(f"first {row['Lat']},{row['Lng']} Hotal is {results[0]['name']}.")
        
        ideal_df.loc[index, 'name'] = results[0]['name']
        ideal_df.loc[index, 'address'] = results[0]['vicinity']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAABse9HaNbruIQ84cFK4MEt7x4d_gWn9q9gIhJdmhhpJAzGm4gjbrCGXVXPdbDHRpQhVOMkfrjXk5Sj6ZecTHXNABqJNNAZ3V1OiANyXiZilH6Viu9Em--sdjEqJS4t0q25RgThYp_hI2OHjZbtikiWWyguCdk_A_aqR2sumy28FSrV7_7vY-TL6oqtK2fexOQh3ybtS1zbHJxLF3buY3pW9zGwEqeWOPGqkvFq-BpRpThKBf-4co0GtLL20xeyTyyEUJEyrDgo7gFF54BANatzLN1e8OVjhkQSyl-5oq_cHjFnqvuSzxmvHOp3BgLrfo9_X3tLoPQjLZVvEXJ0XHE1_vRPIh0GhDEz2WJTG8Zktx30SCh3jvGR2Fp42Thd7bDxIQvk_QZLIhFFAdKz5RqlGeZhoUEDNOY97dQnYSWVpMrqowSgPOhUE",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 27.0368274,
                    "lng": 14.4301373
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.0381662802915,
                        "lng": 14.4314716802915
                    },
                    "southwest": {
                        "lat": 27.0354683197085,
                        "lng": 14.4287737197085

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAANT3QxwnYUNjblBUhT4mcJdnmGKUtZ98RnDqC2T2fdrd2o9xRcqgyhK9LD6hseJW9EjBDXnBJlO_-o_e_s8zLFnReGAqPOWSUfmEUMat5ra3sG1S43GkQEPV-vj45EAZqj_ZrvngV2EVpnO0R9sK6X8Tj3BkhyNEMQ3rTj4mDbNPDYpN5lEfiOewVdCIzHUkFRx5xtiivo-WYPm8L1FkUquL3MtCIKUEbiXz0UUI1_FJdiaBtLofAbLOs_JVFA-G4n5JUQvjTETvakbFciixRv_We8UmzV7p7CjorDPqqonwgeGw8kSMyBJRU3ZW6HnPygzbL_6Aiv-H7f6HYZrZOQ9tzoOdtPeT3X0AqYlSjqKtDlYmgXKPwJ1dLfIFNrzUZRIQBma5aufa0us28MhyKOIgTBoUZRaWUxdG5432XNXTa9g8wHgn5qw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -22.1335949,
                    "lng": 14.2910675
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.1321884697085,
                        "lng": 14.2924805802915
                    },
                    "southwest": {
                        "lat": -22.1348864302915,
                        "lng": 14.2897826197

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -28.4876062,
                    "lng": -48.8397403
                },
                "viewport": {
                    "northeast": {
                        "lat": -28.4862572197085,
                        "lng": -48.8383913197085
                    },
                    "southwest": {
                        "lat": -28.4889551802915,
                        "lng": -48.8410892802915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "06e375a94d644103ca8fa273ff5680a1f62e38ed",
            "name": "Pousada Barra Sul",
            "place_id": "ChIJd4WkCM9RIZURgDnHzlcZhVQ",
            "plus_code": {
                "compound_code": "G566+X4 Sambaqui, Laguna - State of Santa Catarina, Brazil",
                "global_code": "583HG566+X4"

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAHpcG7lj4h4i8DSxUw9MgFBJq7PhjEe3VM0rF-RYt5F7GkyRwKkjfXHA9zPewPwvdnPoKY-N_yF9ofEz95N-M75Q7eF2zAp_04R-QHGemEdYkWSNHVZPE79PtVRX7hOdBkHtX8rMAmjcPg9Gnt5Tp1Y2PV9dZxCzsZIM_-XmrwoGWc4s0pzsXjRkFIuQfg8an0-_22t5SixYdEHa0bJDzPEvUTHidkw3NJiohrpUd2e1eaA0IcEdtIU7LtdHYzLGYFOhHkoilFp4I_DwVj0joHVWReILml-RSfeRl3SkObSkhcdqdJ77QQcvQDGqY-lD7CBELv_eon6FKNXViE1BIiGh8IlGphoMoEeNVvtDQ8hX1MGnfwwLeYgM8FIS3Rn2ZBIQYC86rIVw-mmtXHdonakQXBoUrjcgB80IkUBICrQF3ggDOjUPAwo",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 28.0916205,
                    "lng": -16.7404893
                },
                "viewport": {
                    "northeast": {
                        "lat": 28.09316360000001,
                        "lng": -16.7388539
                    },
                    "southwest": {
                        "lat": 28.08975479999999,
                        "lng": -16.7420707
      

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 29.040621,
                    "lng": 21.4991334
                },
                "viewport": {
                    "northeast": {
                        "lat": 29.0462739,
                        "lng": 21.5023764
                    },
                    "southwest": {
                        "lat": 29.0387367,
                        "lng": 21.4980524
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "51d75a13e7c6e3014b917a8118ece7ffe9464528",
            "name": "Dr. Mohsen Villa",
            "place_id": "ChIJI0xj9nMfiRMRRoGJxXHnuV4",
            "plus_code": {
                "compound_code": "2FRX+6M Jalu, Libya",
                "global_code": "7GX32FRX+6M"
            },
            "reference": "ChIJI0xj9nMfiRMRRoGJxXHnuV4",

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAJYFRcgRHhc_qsWwGX5VPYhd40pd7a_y11OFzVDzsY_mdvznhsyS8XPGctCtyQdftBwNhqD6XiqWV3JzIxEsRoCIWzoMZECFDEHJ5mTs6YSeLAFNew0oYM2J3tz1G4Qk4TbGTKMkNr4XpwVvw0SsqygPk2pz0Ybwurc7h23QB0hRtYBltvDwQXlKYiFi9Q4SN_fni43dX-kxC8BwQyo0jQgnaSZ-g-1iCByP36Yx41GgZrOw0DjHozWkR5cg8_tz3D5EeTCbRmQTcugzu-DymHT3ByoAOsEpNnLoB4yaHOVCZioeryIG00BASlZj0xc0H64E4Sh9unXRbTaL1cLwP_bXEbUPLFkFAgFOiB4BV6au1qUYRIllTxgF0tSI_OJwVhIQl4UKKgNHtIQL_eqAwtFE8xoUiB6WehYYF64DFgG7zktmiiWaCO8",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -42.7166176,
                    "lng": 170.9621887
                },
                "viewport": {
                    "northeast": {
                        "lat": -42.7153028197085,
                        "lng": 170.9635860802915
                    },
                    "southwest": {
                        "lat": -42.7180007802915,
                        "lng": 170.9608881

In [238]:
 ideal_df.reset_index()

,index,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (m/s),Wind Speed (mph),Lat,Lng,name,address
0,37,sabha,ly,50.74,24.0,0.0,3.18,7.113469,27.036541,14.429024,محطة تاكسيات الشاطئ براك,محطة سبها، سبها،
1,194,henties bay,na,65.32,83.0,0.0,2.01,4.496249,-22.113496,14.283204,Fishermans Guest House,Namibia
2,220,laguna,br,64.42,82.0,0.0,2.26,5.055484,-28.487606,-48.839740,Pousada Barra Sul,State of Santa Catarina
3,265,qom,ir,55.78,19.0,0.0,1.59,3.556735,34.641576,50.874603,Qom International Hotel,"Qom, Eram Street"
4,272,adeje,es,63.21,63.0,0.0,3.36,7.516118,28.122060,-16.733821,Gran Tacande Wellness & Relax Costa Adeje,"C/Alcalde Walter Paetzmann, s/n, Adeje"
5,312,jalu,ly,55.60,68.0,0.0,4.07,9.104346,29.040621,21.499133,Dr. Mohsen Villa,Jalu
6,501,hokitika,nz,63.70,71.0,0.0,3.29,7.359533,-42.717369,170.966543,Beachfront Hotel Hokitika,"111 Revell Street, Hokitika"


In [239]:
# Store 'Lat' and 'Lng' into  locations 
ideal_locations = ideal_df[["Lat", "Lng"]].astype(float)
ideal_humidity = ideal_df["Humidity (%)"].astype(float)
ideal_df.head()
hotel_df=ideal_df[["name","City","Country"]]
hotel_df['location'] =ideal_df['Lat'].map(str)+','+ideal_df['Lng'].map(str)
hotel_df.to_numpy()
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(ideal_locations, weights=ideal_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

marker_locations=hotel_df["location"]
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel["name"]}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(*hotel) for hotel in hotel_df]
markers = gmaps.marker_layer(marker_locations,info_box_content=hotel_info)
fig.add_layer(markers)

fig

/Users/jadetao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


KeyError: 'hotel'

In [240]:
hotel_df

,name,City,Country,location
37,محطة تاكسيات الشاطئ براك,sabha,ly,"27.0365406,14.4290236"
194,Fishermans Guest House,henties bay,na,"-22.1134964,14.2832038"
220,Pousada Barra Sul,laguna,br,"-28.4876062,-48.8397403"
265,Qom International Hotel,qom,ir,"34.6415764,50.8746035"
272,Gran Tacande Wellness & Relax Costa Adeje,adeje,es,"28.1220604,-16.7338211"
312,Dr. Mohsen Villa,jalu,ly,"29.040621,21.4991334"
501,Beachfront Hotel Hokitika,hokitika,nz,"-42.7173686,170.9665427"
